# ICICI type 2

In [2]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO

In [12]:


def abc(a):
    if type(a) ==str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else :
        z=a
    return z

def kotak(f):
    tables = camelot.read_pdf(f, flavor='stream', pages='all')

    if len(tables) != 0:
        dt = pd.DataFrame(); tmp = pd.DataFrame();
        for i in range(len(tables)):
            tmp = tables[i].df
            if (tmp.shape[1]<5):
                pass
            elif (tmp.shape[1]==5):
                try:
                    idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('deposit').any(), axis=1) ==True].index ][0]
                    tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True)
                except:
                    print("\nAxis-Column headers missing"); pass;
                tmp.columns = ["Date","Narration","Chq/Ref No","Withdrawal(Dr)/Deposit(Cr)","Balance"]
                dt=pd.concat([dt,tmp]).reset_index(drop=True)
            else:
                print("\nDifferent structure found"); pass;

    dt = dt[~dt.index.isin(dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c|statement summary|generated on|generated by|computer generated statement|not').any(), axis=1) ==True].index)]
    dt.drop(dt.nunique(dropna=False)[(dt.nunique(dropna=False) == 1)].index, axis=1,inplace=True)             
    dt.iloc[:,-1]=dt.iloc[:,-1].fillna(dt.iloc[:,-2])
    
    try:
        bal=[c for c in dt.columns if "BALANCE" in str(c).upper() ][0]
        dt[bal]=dt[bal].apply( lambda x: abc(x) )
    except: print("\nBalance columns missing")

    dt['flag']=dt.iloc[:,-1].shift(1)
    dt=dt[~dt.index.isin(dt[dt.iloc[:,-1]==dt.iloc[:,-2]].index)]

    try:
        amt=[c for c in dt.columns if "WITHDRAW" in str(c).upper() ][0]
    except:
        try:
            amt=[c for c in dt.columns if "DEPOSIT" in str(c).upper() ][0]
        except: pass   

    dt[[bal,amt]]=dt[[bal,amt]].replace(r'^\s*$', np.nan, regex=True)

    try:
        dt[amt]=dt[amt].apply( lambda x: x.split(' ')[0] if type(x) == str else x )                       
        dt['Wdl1']=dt[amt].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float) *-1               
    except Exception as e: 
        print("Error",e)
 

    dt[bal]=dt[bal].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)                                 
    dt['flag']=dt.iloc[:,0].astype(str)+dt['Wdl1'].astype(str) +dt[bal].astype(str)
    dt['flag2']=np.arange(len(dt))

    dt.loc[ dt[['flag']].duplicated(keep=False) , 'flag' ] =dt['flag'] + dt['flag2'].astype(str)  
    dt['flag']=dt['flag'].apply( lambda row : np.nan if 'nannan' in row else row ).fillna(method='ffill')        
    dt['Narration']=dt.groupby('flag')['Narration'].transform( lambda x :''.join(x)) 
    dt['Chq/Ref No']=dt.groupby('flag')['Chq/Ref No'].transform( lambda x :''.join(x))
    dt = dt.drop_duplicates(['flag'], keep='first').iloc[0:,:-3].reset_index(drop=True)

    dt["Debits"] = np.nan; dt["Credits"] = np.nan;
    for j,i in enumerate(dt[amt]):
        if "Cr" in str(i):
            dt["Credits"][j] = dt[amt][j]
        elif "Dr" in str(i):
            dt["Debits"][j] = dt[amt][j]

    dt["Debits"]=dt["Debits"].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)*-1
    dt["Credits"]=dt["Credits"].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)

    try:
        dat=[c for c in dt.columns if "TRANSACTION DATE" in str(c).upper() ][0]
    except:
        try:
            dat=[c for c in dt.columns if "TXN DATE" in str(c).upper() ][0]
        except:
            try:
                dat=[c for c in dt.columns if "DATE" in str(c).upper() ][0]
            except:pass

    try:
        chq=[c for c in dt.columns if "CHQ" in str(c).upper() ][0]
    except:
        try:
            chq=[c for c in dt.columns if "CHEQUE" in str(c).upper() ][0]
        except:pass

    try:
        narr=[c for c in dt.columns if "REMARKS" in str(c).upper() ][0]
    except:
        try:
            narr=[c for c in dt.columns if "PARTICULARS" in str(c).upper() ][0]
        except:
            try:
                narr=[c for c in dt.columns if "DESCRIPTION" in str(c).upper() ][0]
            except:
                try:
                    narr=[c for c in dt.columns if "DETAILS" in str(c).upper() ][0]
                except:
                    try:
                        narr=[c for c in dt.columns if "NARRATION" in str(c).upper() ][0]
                    except:pass

    try:
        deb=[c for c in dt.columns if "DEBIT" in str(c).upper() ][0]
    except:
        try:
            deb=[c for c in dt.columns if "AMOUNT" in str(c).upper() ][0]
        except:
            try:
                deb=[c for c in dt.columns if "WITHDRAW" in str(c).upper() ][0]
            except:pass

    try:
        crd=[c for c in dt.columns if "CREDIT" in str(c).upper() ][0]
    except:
        try:
             crd=[c for c in dt.columns if "DEPOSIT" in str(c).upper() ][0]
        except:pass

    dt = dt[[dat,chq,narr,deb,crd,bal]]
    dt.columns = ["Xns Date","Cheque No","Narration","Debits","Credits","Balance"]  
    
    dt.drop_duplicates(keep='first', inplace=True)
    
    dt.to_excel("check.xlsx", index=0)
    print("parsed")
    return dt

f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\KOTAK\files\type2\kotak20.pdf'
df = kotak(f)

parsed


c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_g

In [39]:
df.head()

,Xns Date,Cheque No,Narration,Debits,Credits,Balance
0,,,B/F,NaN,0.0,54914.2
1,01-04-2018,809117223568,PCD/2000688772/D MART/MUMBAI,-1128.4,NaN,53785.8
2,01-04-2018,IMPS-809201324215,MB PULSE Ref809201968402(Value Da...,-2000.0,NaN,51785.8
3,01-04-2018,IMPS-809201324314,MB PULSE Ref809201968454(Value Da...,-3358.0,NaN,48427.8
4,01-04-2018,IMPS-809201324541,MB PULSE Ref809201968584(Value Da...,-4000.0,NaN,44427.8


In [18]:
try:
    amt=[c for c in df.columns if "WITHDRAW" in str(c).upper() ][0]
except:
    try:
        amt=[c for c in df.columns if "DEPOSIT" in str(c).upper() ][0]
    except: pass

In [21]:
df["Debits"] = np.nan
df["Credits"] = np.nan

In [22]:
for j,i in enumerate(df[amt]):
    if "Cr" in str(i):
        df["Credits"][j] = df[amt][j]
    elif "Dr" in str(i):
        df["Debits"][j] = df[amt][j]

c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the 

In [28]:
df["Debits"]=df["Debits"].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)*-1
df["Credits"]=df["Credits"].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)

In [29]:
df.head()

,Date,Narration,Chq/Ref No,Withdrawal(Dr)/Deposit(Cr),Balance,Debits,Credits
0,,B/F,,-0.0,54914.25,NaN,0.0
1,01-04-2018,PCD/2000688772/D MART/MUMBAI,809117223568,-1128.4,53785.85,-1128.4,NaN
2,01-04-2018,MB PULSE Ref809201968402(Value Da...,IMPS-809201324215,-2000.0,51785.85,-2000.0,NaN
3,01-04-2018,MB PULSE Ref809201968454(Value Da...,IMPS-809201324314,-3358.0,48427.85,-3358.0,NaN
4,01-04-2018,MB PULSE Ref809201968584(Value Da...,IMPS-809201324541,-4000.0,44427.85,-4000.0,NaN
